# 数据载入

In [235]:
import pandas as pd
import os
from collections import Counter
import numpy as np

import torch

In [236]:
pd.set_option('display.max_colwidth', -1)

C:\Users\c'y'y\AppData\Local\Temp\ipykernel_48640\3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## poi数据（全面）

In [237]:
poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ISO-8859-1')
# poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ANSI')
poi_df = pd.DataFrame(poi_data)

In [238]:
poi_df

,FID_points,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,...,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,POINT_X,POINT_Y
0,25302,9652365665,Naval Maritime Academy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.17_1,Mumbai City,NaN,NaN,District,District,NaN,IN.MH.MC,72.810538,18.895218
1,14850,5846473787,Indian Meterological Department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.17_1,Mumbai City,NaN,NaN,District,District,NaN,IN.MH.MC,72.813203,18.897915
2,12449,4583581092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.17_1,Mumbai City,NaN,NaN,District,District,NaN,IN.MH.MC,72.813697,18.899786
3,14668,5729410236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.17_1,Mumbai City,NaN,NaN,District,District,NaN,IN.MH.MC,72.814971,18.900186
4,25401,9739328478,INS Ashwini Hospital,NaN,bus_stop,NaN,NaN,NaN,NaN,NaN,...,IND.20.17_1,Mumbai City,NaN,NaN,District,District,NaN,IN.MH.MC,72.814360,18.900772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27141,33202,11056076828,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.18_1,Mumbai Suburban,NaN,NaN,District,District,NaN,IN.MH.MC,72.873904,19.265257
27142,13116,4910114922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.18_1,Mumbai Suburban,NaN,NaN,District,District,NaN,IN.MH.MC,72.873672,19.265771
27143,5992,2980188909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.18_1,Mumbai Suburban,NaN,NaN,District,District,NaN,IN.MH.MC,72.856508,19.266814
27144,7324,3230936705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,IND.20.18_1,Mumbai Suburban,NaN,NaN,District,District,NaN,IN.MH.MC,72.857539,19.266971


In [239]:
poi_df.columns

Index(['FID_points', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags', 'FID_Mumbai', 'GID_0',
       'NAME_0', 'GID_1', 'NAME_1', 'NL_NAME_1', 'GID_2', 'NAME_2',
       'VARNAME_2', 'NL_NAME_2', 'TYPE_2', 'ENGTYPE_2', 'CC_2', 'HASC_2',
       'POINT_X', 'POINT_Y'],
      dtype='object')

In [ ]:
# save tag list to txt

poi_df_nec=poi_df.loc[:,['FID_points', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags','NAME_2', 'POINT_X', 'POINT_Y']]
poi_df_ot =poi_df_nec.loc[:,['other_tags']]
tag_list = []
def collect_tags(x):
    # print(x['other_tags'])
    
    if  pd.notna(x['other_tags']):
        # print(x['other_tags'])
        tags = x['other_tags'].split("\",\"")
        # print(tags)
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                if t not in tag_list:
                    tag_list.append(t)
            except:
                pass
    # return tag_list

result = poi_df_ot.apply(lambda x: collect_tags(x), axis=1, result_type='expand')
# result
tag_list.sort()
tag_list
# 保存tag_list到txt
with open(r'E:\xpj\research\POI\tag_list.txt', 'w') as f:
    for item in tag_list:
        f.write("%s\n" % item)

# poi_df_ot

# def genSent(x):
#     tags = None
#     b = True    
#     # sen = "The imformation of this point contains that "
#     sen = "Nearby,"
#     for key in ['name','barrier', 'highway','address','place','man_made']:
#         if not x[key].isna().iloc[0]:
#             # print(x[key])
#             sen = sen + key +" is " + x[key].iloc[0] + ","
#     if not x['other_tags'].isna().iloc[0]:
#         tags = x['other_tags'].iloc[0].split("\",\"")
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 k = tag.split("=>")[1]
#                 if "name:" in t and t != "name:en":
#                     pass
#                 sen = sen + t +" is " + k + ","
#             except:
#                 pass

#     sen = sen + "located in " + x['NAME_2'].iloc[0]
    
#     # return sen,b
    
#     return sen
# # poi_df_nec


## 照片信息

In [240]:
pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ISO-8859-1')
# pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ANSI')
pic_df = pd.DataFrame(pic_data)

In [241]:
pic_df

,img_key,captured_a,camera_mak,created_at,sequences_,pano,user_key,username,ca,lon,...,nearpoi_1,dis_1,nearpoi_2,dis_2,nearpoi_3,dis_3,nearpoi_4,dis_4,nearpoi_5,dis_5
0,7IQaA8gTA8eF3j010TpluA,2015/12/25 19:03:45.000,ASUS,2015/12/27 20:22:04.000,Cm2iMoD2Gh0CAO9RJMHh5g,0,bhbwfMdubPMiem62NcMRow,jnikheel,218.91900,72.976903,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,f4aEv-WMYHvao2qTUOjA9A,2017/12/26 07:11:19.000,OPPO,2017/12/26 10:59:26.000,PjFW5wnrAjREedgLjI7JUg,0,o5A4ZYvAV65VhlMAQGc66g,shalu,0.00000,72.997034,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,M1xlI3rE7Aa6iZAiEtLPxw,2014/12/25 14:05:34.000,XIAOMI,2014/12/25 15:29:14.000,9_c7z16hlkTCJyktdXEDlA,0,t9AKi036Mnbjr0tqe2hU6Q,nikhil27,7.30147,72.834905,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,IjZCnm61iEVs2HPWgMnT4g,2015/12/25 19:03:45.000,ASUS,2015/12/27 20:22:04.000,Cm2iMoD2Gh0CAO9RJMHh5g,0,bhbwfMdubPMiem62NcMRow,jnikheel,229.67400,72.976875,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,YyTflWvHnXeP1Xp2-9mszA,2015/12/25 19:03:45.000,ASUS,2015/12/27 20:22:04.000,Cm2iMoD2Gh0CAO9RJMHh5g,0,bhbwfMdubPMiem62NcMRow,jnikheel,231.40700,72.976871,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39272,6k10dCvHvUmQxguHAH7EVA,NaN,NaN,NaN,NaN,0,NaN,NaN,0.00000,72.872900,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
39273,EQqnJTYm6w18GpvtqycPDA,NaN,NaN,NaN,NaN,0,NaN,NaN,0.00000,72.872900,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
39274,viuutfmeUaVAf78FjFMb2g,NaN,NaN,NaN,NaN,0,NaN,NaN,0.00000,72.872900,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
39275,ZLLdLiKfJx6hl0tXQscYVQ,NaN,NaN,NaN,NaN,0,NaN,NaN,0.00000,72.872900,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


## 照片poi关联

In [242]:
pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ISO-8859-1')
# pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ANSI')
pic_poi_df = pd.DataFrame(pic_poi_data)

In [243]:
pic_poi_df

,img_key,lon,lat,nearpoi_1,dis_1,nearpoi_2,dis_2,nearpoi_3,dis_3,nearpoi_4,dis_4,nearpoi_5,dis_5
0,7IQaA8gTA8eF3j010TpluA,72.976903,19.194708,21511,14.197669,24840,14.210550,21512,14.294612,20475,14.332980,20497,14.492412
1,f4aEv-WMYHvao2qTUOjA9A,72.997034,19.102010,2097,54.865569,2098,55.042855,2099,55.123294,6367,55.229985,6364,55.329367
2,M1xlI3rE7Aa6iZAiEtLPxw,72.834905,18.941158,25439,0.440531,707,0.595962,3896,0.622014,2179,0.655078,4014,0.680846
3,IjZCnm61iEVs2HPWgMnT4g,72.976875,19.194667,21511,14.152732,24840,14.163299,21512,14.249661,20475,14.286303,20497,14.443676
4,YyTflWvHnXeP1Xp2-9mszA,72.976871,19.194651,21511,14.136111,24840,14.146671,21512,14.233040,20475,14.269645,20497,14.427363
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39272,6k10dCvHvUmQxguHAH7EVA,72.872900,19.061500,20632,0.347828,27778,0.348925,28996,0.639271,24700,0.640030,34691,0.648738
39273,EQqnJTYm6w18GpvtqycPDA,72.872900,19.062600,7877,0.318658,27779,0.329388,34691,0.495057,24700,0.526287,28996,0.613993
39274,viuutfmeUaVAf78FjFMb2g,72.872900,19.062700,7877,0.282600,27779,0.287674,34691,0.590848,24700,0.620418,340,0.700574
39275,ZLLdLiKfJx6hl0tXQscYVQ,72.872900,19.070200,8351,0.313712,664,1.006733,5729,1.065979,28263,1.110924,1432,1.176598


## 照片描述

In [ ]:
desc_pd = pd.DataFrame(columns=['image','year','prompt'])

In [ ]:
desc_path = r'E:\xpj\dataset\clip\desc'  #打开桌面位置处的文件夹hello，注意格式\\
result=os.listdir(desc_path)

for f in result:
    
    year = int(f[4:8])
    temp_df = pd.read_csv(desc_path+'\\'+f ,encoding='ISO-8859-1')
    temp_df['year'] = year
    desc_pd = pd.concat([desc_pd,temp_df],axis=0)

In [ ]:
desc_pd.to_csv(r"E:\xpj\research\POI\desc.csv")
desc_pd

# 数据处理

## poi数据预处理

In [ ]:
poi_df_nec=poi_df.loc[:,['FID_points', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags','NAME_2', 'POINT_X', 'POINT_Y']]

In [ ]:
poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec.csv")

# 硬提示计算相似度，找最佳poi

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 加载BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model = AutoModel.from_pretrained(r"E:\xpj\models\bert-base-uncased")

In [ ]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)
    # print(inputs)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim


In [ ]:

def isEnglish(s):
    if isinstance(s, str):
        return s.isascii()
    return False



# 从txt中读取tag_list
tag_ignore_list = []
with open(r'E:\xpj\research\POI\tag_ignore_list.txt', 'r') as f:
    for line in f.readlines():
        tag_ignore_list.append(line.strip())
# print(tag_ignore_list)

def genSent(x):
    tags = None
    b = True    
    # sen = "The imformation of this point contains that "
    sen = "Nearby,"
    for key in ['name','barrier', 'highway','address','place','man_made']:
        # if isinstance(x[key],str) and not isEnglish(x[key]):
        #     continue
        if key == 'name' and not isEnglish(x['name'].iloc[0]):
            continue
        if not x[key].isna().iloc[0]:
            # print(x[key])
            sen = sen + key +" is " + x[key].iloc[0] + ","
    # print(sen)
    if not x['other_tags'].isna().iloc[0]:
        tags = x['other_tags'].iloc[0].split("\",\"")
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                k = tag.split("=>")[1]
                
                if not isEnglish(k) or not isEnglish(t):
                    # print(k,t)
                    continue
                if list(t)[-1] == "=":
                    continue
                if t in tag_ignore_list:
                    continue
                # if "name:" in t and t != "name:en":
                #     pass
                # pass 的作用是什么也不做 跳过，continue的作用是执行下次for循环
                # print(t,k)
                sen = sen + t +" is " + k + ","
            except:
                pass

    sen = sen + "located in " + x['NAME_2'].iloc[0]
    
    # return sen,b
    
    return sen

In [ ]:
# a="脿陇陇脿陇戮脿陇聹脿陇庐脿陇鹿脿陇戮脿陇脿陇陋脿楼聟脿陇虏脿楼聡脿陇脿陇鹿脿楼聣脿陇聼脿楼聡脿陇"
# # print(isEnglish(a))
# b="à¤­à¤¾à¤°à¤¤à¥à¤à¤¹à¤µà¤¾à¤®à¤¾à¤¨ à¤µà¤¿à¤­à¤¾à¤"
# print(isEnglish(b))
# print(b.encode('utf-8').decode('ISO-8859-1'))
s = "啶ぞ啶班い啷€啶啶灌さ啶距ぎ啶距え 啶掂た啶ぞ啶"
print(s.encode('ansi').decode('iscii'))

将poi对应的自然语言提示存储到文件中(poi_nec最后插入一列提示)

In [ ]:
def store_poi(x):
    # print(x)
    poi = x[0]
    # print(poi)    
    poi_im = poi_df_nec.loc[poi_df_nec['FID_points']==poi]
    sen = genSent(poi_im)
    return sen
    # print(sen)


    

In [ ]:
from tqdm import tqdm
result =[]
for index, row in tqdm(poi_df_nec.iterrows(), total=poi_df_nec.shape[0], desc="Processing"):
    # if index == 130:
    #     print(store_poi(row))
    result.append(store_poi(row))
# result = poi_df_nec.apply(lambda x: store_poi(x), axis=1, result_type='expand')
# poi_df_nec['poi_prompt']= result
# poi_df_nec['poi_prompt'] = result

In [ ]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec3.csv")
poi_df_nec['poi_prompt'] = result
poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_prompt3.csv")
poi_df_nec

In [ ]:
def find_best_poi(x):
    # print(x)
    img_key = x['img_key']
    img_file = img_key+'.jpg'
    try:
        img_desc = desc_pd.loc[desc_pd['image']==img_file]['prompt'].iloc[0]
    except:
        print("The description of picture " + img_file +" doesn't exist")
        return  None,None
    pois = x[3::2]
    pois_dis =x[4::2]
    # print(pois)
    similarities = []
    sentences = []
    for index,poi in enumerate(pois):
        # 设置距离阈值
        if pois_dis[index]<1:
            poi_im = poi_df_nec.loc[poi_df_nec['FID_points']==poi]
            sen = genSent(poi_im)
        
            similarity = calc_similarity(img_desc,sen)
            sentences.append(sen)
            similarities.append(similarity)
        
    if len(similarities) == 0:
        best_poi=None
        return best_poi,0
    elif similarities and max(similarities)>0.5:
        best_poi = pois[similarities.index(max(similarities))]
        return best_poi,max(similarities)

In [ ]:
result = []
for _, row in tqdm(pic_poi_df.iterrows(), total=pic_poi_df.shape[0], desc="Processing"):
    result.append(find_best_poi(row))

pic_poi_df[['best_poi', 'best_poi_score']] = result

In [ ]:
result = pic_poi_df.apply(lambda x: find_best_poi(x), axis=1,result_type='expand')

In [ ]:
pic_poi_df[['best_poi','similarity']]= result

In [ ]:
pic_poi_df

In [ ]:
pic_poi_df.to_csv(r"E:\xpj\research\POI\\best_poi2.csv")

In [ ]:
df1 = pd.merge(pic_poi_df, poi_df_nec, left_on="best_poi",right_on = 'FID_points',how="left")

In [ ]:
df1

In [ ]:
df1.to_csv(r"E:\xpj\research\POI\\best_poi_information2.csv")

In [ ]:
df1 = pd.read_csv(r"E:\xpj\research\POI\best_poi_information2.csv")
poi_df_nec.columns

In [ ]:
# desc_pd: file,prompt insert poi_prompt
# df1 : image_key, bestpoi
# poi_df_nec: FID ,poi_prompt
def connect(x):
    image_file_name = x[0]
    image_name =image_file_name.split('.')[0]
    best_poi_row = df1.loc[df1['img_key']==image_name]['best_poi']
    best_poi = None
    if best_poi_row.notna().any():
        best_poi=int(best_poi_row.iloc[0])
        poi_prompt = poi_df_nec.loc[poi_df_nec['FID_points']==best_poi]['poi_prompt'].iloc[0]
        return poi_prompt
        # print(poi_prompt)
    # print(best_poi)
    # print("\n")
    

    # print(image_name)

In [ ]:
result2 = desc_pd.apply(lambda x: connect(x), axis=1,result_type='expand')
result2

In [ ]:
desc_pd['poi_prompt'] = result2

In [ ]:
desc_pd

In [ ]:
desc_pd.to_csv(r"E:\xpj\research\POI\desc_poi2.csv")